In [1]:
import torch
import torch.nn as nn
import matplotlib.pyplot as plt
import numpy as np

from src.autoencoder import Autoencoder, VariationalAutoEncoder
from utils.mnist_loader import data_download, data_loader
from utils.model_trainer import autoencoder_trainer, vae_trainer
from utils.visualization import visualization

In [2]:
USE_CUDA = torch.cuda.is_available()
DEVICE = torch.device("cuda:0" if USE_CUDA else "cpu")
EPOCHS = 100
SAMPLES = 5
print(DEVICE)
train_data, test_data = data_download()
train_loader, test_loader = data_loader(train_data, test_data, batch_size=256)

cuda:0
number of training data :  60000
number of test data :  10000


# Autoencoder

In [3]:
ae=Autoencoder(n_hidden=336, z_dim=128).to(DEVICE)
criteria = nn.MSELoss()
optimizer = torch.optim.Adam(ae.parameters(), lr=0.001)

In [4]:
train_loss, test_loss = autoencoder_trainer(model=ae, 
                                            criteria=criteria, optimizer=optimizer, 
                                            train_loader=train_loader, test_loader=test_loader, 
                                            device=DEVICE, epochs = EPOCHS)

  1%|          | 1/100 [01:01<1:41:09, 61.31s/it]

epochs: 1 - Train loss: 0.05994917452335358 - Test loss: 0.06586161255836487


  2%|▏         | 2/100 [01:50<1:28:03, 53.91s/it]

epochs: 2 - Train loss: 0.04470355436205864 - Test loss: 0.05165022984147072


  3%|▎         | 3/100 [02:41<1:25:42, 53.02s/it]

...

epochs: 10 - Train loss: 0.01216402929276228 - Test loss: 0.009272782132029533


 11%|█         | 11/100 [10:15<1:30:26, 60.97s/it]

epochs: 11 - Train loss: 0.012111456133425236 - Test loss: 0.011633534915745258


 12%|█▏        | 12/100 [11:14<1:28:37, 60.43s/it]

epochs: 12 - Train loss: 0.010128813795745373 - Test loss: 0.012046068906784058


In [ ]:
visualization(loader=test_loader, model=ae, device=DEVICE, num_of_samples=SAMPLES)

In [ ]:
plt.plot(np.array(test_loss))
plt.show()

# Variational Autoencoder

In [ ]:
def vae_loss(reconstruction, x, mu, log_var):
    reconstruction_loss = nn.functional.binary_cross_entropy(reconstruction, x, reduction='sum') # bernoulli distribution assumption
    kl_loss = - 0.5 * torch.sum(1 + log_var - mu.pow(2) - log_var.exp())
    return reconstruction_loss, kl_loss

In [ ]:
vae = VariationalAutoEncoder(n_hidden=336, z_dim=128).to(DEVICE)
optimizer = torch.optim.Adam(vae.parameters(), lr=0.001)
criteria = vae_loss

In [ ]:
train_loss, test_loss = vae_trainer(model=vae, beta=1,
                                    criteria=criteria, optimizer=optimizer, 
                                    train_loader=train_loader, test_loader=test_loader, 
                                    device=DEVICE, epochs = EPOCHS)

In [ ]:
visualization(loader=test_loader, model=vae, device=DEVICE, num_of_samples=SAMPLES)

In [ ]:
vae.cpu()
generated_samples = vae.generate(SAMPLES)

for sample in generated_samples:
    plt.matshow(sample.reshape(28,28))
    plt.show()

In [ ]:
plt.plot(np.array(test_loss))
plt.show()

# Beta-Variation Autoencoder

In [ ]:
b_vae = VariationalAutoEncoder(n_hidden=336, z_dim=128).to(DEVICE)
optimizer = torch.optim.Adam(b_vae.parameters(), lr=0.001)
criteria = vae_loss

In [ ]:
train_loss, test_loss = vae_trainer(model=b_vae, beta=4,
                                    criteria=criteria, optimizer=optimizer, 
                                    train_loader=train_loader, test_loader=test_loader, 
                                    device=DEVICE, epochs = EPOCHS)

In [ ]:
visualization(loader=test_loader, model=b_vae, device=DEVICE, num_of_samples=SAMPLES)

In [ ]:
b_vae.cpu()
generated_samples = b_vae.generate(SAMPLES)

for sample in generated_samples:
    plt.matshow(sample.reshape(28,28))
    plt.show()

In [ ]:
plt.plot(np.array(test_loss))
plt.show()